# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = '../WeatherPy/output_data/cities.csv'
weather_df = pd.read_csv(file)
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Busselton,-33.65,115.33,62.01,59,0,1.99,AU,1606265562
1,Albany,42.60,-73.97,33.01,61,99,2.55,US,1606265562
2,Port Elizabeth,-33.92,25.57,59.00,77,0,11.41,ZA,1606265562
3,Nampula,-15.12,39.27,77.00,69,75,12.75,MZ,1606265562
4,Zhezkazgan,47.80,67.71,9.32,96,84,10.13,KZ,1606265563


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
fig = gmaps.figure()
locations = weather_df[['Lat','Lng']]
humidities = weather_df['Humidity']
heatmap = gmaps.heatmap_layer(locations, humidities)
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
hotel_df = weather_df.loc[(weather_df['Max Temp'] < 85) & (weather_df['Max Temp'] > 75) & (weather_df['Humidity'] < 70) & (weather_df['Wind Speed'] < 10) & (weather_df['Cloudiness'] == 0), :]

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,Victor Harbor,-35.57,138.62,84.00,35,0,9.28,AU,1606265564
28,Port Lincoln,-34.73,135.87,79.48,33,0,7.61,AU,1606265429
99,Broken Hill,-31.95,141.43,80.60,26,0,8.05,AU,1606265581
123,Acajutla,13.59,-89.83,77.00,65,0,6.02,SV,1606265585
242,Umluj,25.02,37.27,75.18,58,0,3.18,SA,1606265610
264,Cerro Largo,-28.15,-54.74,84.00,46,0,7.07,BR,1606265615
344,El Calvario,14.12,-89.92,77.00,65,0,5.68,GT,1606265632
429,San Juan,-31.54,-68.54,77.97,27,0,7.54,AR,1606265648


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df['Hotel Name']=''
hotel_df.loc[:, 'Hotel Name'] ==''
hotel_df.head()

C:\Users\Ryan\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
10,Victor Harbor,-35.57,138.62,84.00,35,0,9.28,AU,1606265564,
28,Port Lincoln,-34.73,135.87,79.48,33,0,7.61,AU,1606265429,
99,Broken Hill,-31.95,141.43,80.60,26,0,8.05,AU,1606265581,
123,Acajutla,13.59,-89.83,77.00,65,0,6.02,SV,1606265585,
242,Umluj,25.02,37.27,75.18,58,0,3.18,SA,1606265610,


In [6]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
   
    name_address = requests.get(base_url, params=params).json()
    name = name_address["results"][1]["name"]
    hotel_df.loc[index, "Hotel Name"] = name
    hotel_df.loc[index, "Hotel Name"] == name


C:\Users\Ryan\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [9]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
10,Victor Harbor,-35.57,138.62,84.00,35,0,9.28,AU,1606265564,Apollon Motor Inn
28,Port Lincoln,-34.73,135.87,79.48,33,0,7.61,AU,1606265429,Port Lincoln Hotel
99,Broken Hill,-31.95,141.43,80.60,26,0,8.05,AU,1606265581,Daydream Motel & Cottages
123,Acajutla,13.59,-89.83,77.00,65,0,6.02,SV,1606265585,"Constructora Umaña, S.A."
242,Umluj,25.02,37.27,75.18,58,0,3.18,SA,1606265610,Rwnza Hotel Apartments
264,Cerro Largo,-28.15,-54.74,84.00,46,0,7.07,BR,1606265615,Lojas Becker
344,El Calvario,14.12,-89.92,77.00,65,0,5.68,GT,1606265632,Mancomunidad El Pacífico
429,San Juan,-31.54,-68.54,77.97,27,0,7.54,AR,1606265648,Hotel Provincial


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations1 = hotel_df[["Lat", "Lng"]]

In [11]:
markers1 = gmaps.marker_layer(locations1, info_box_content=hotel_info)
fig.add_layer(markers1)
fig

Figure(layout=FigureLayout(height='420px'))